In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
with open("sleeping_beauty.txt", 'r') as f:
    lines = f.readlines()

all_story = [line.strip() for line in lines]

idx = 0
document = all_story[idx]

# LSA Summarizer

In [32]:
from lsa_summarizer import LsaSummarizer

lsa = LsaSummarizer()
lsa(document, 1)

('A king and queen once upon a time reigned in a country a great way off, where there were in those days fairies.',)

# Transformer

In [33]:
from pretrained_summarizer import T5StorySumSummarizer

t5 = T5StorySumSummarizer()
t5(document)

Your max_length is set to 512, but you input_length is only 97. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=48)


[{'summary_text': 'A King and Queen Once upon a Time Reigned In A Country A great Way Off, Where there were in these days fairies'}]

In [34]:
from pretrained_summarizer import BartSummarizer

bart_cnn = BartSummarizer('cnn')
bart_cnn(document)

Your max_length is set to 142, but you input_length is only 89. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)


[{'summary_text': 'A king and queen once upon a time reigned in a country a great way off, where there were in those days fairies. Though they had been married many years they had no children, and this grieved them very much indeed. They had plenty of money, and plenty of fine clothes to wear, and lots of good things to eat and drink.'}]

In [35]:
from pretrained_summarizer import BartSummarizer

bart_xsum = BartSummarizer('xsum')
bart_xsum(document)

[{'summary_text': 'A long time ago there was a king and queen, and they had no children, and this grieved them very much indeed.'}]

In [36]:
from pretrained_summarizer import DistilBartSummarizer

dbart_xsum = DistilBartSummarizer('xsum-12-3')
dbart_xsum(document)

[{'summary_text': ' X-Men is the story of a king and queen who have no children.'}]

In [2]:
from lsa_summarizer import LsaSummarizer
from pretrained_summarizer import T5StorySumSummarizer
from pretrained_summarizer import BartSummarizer
from pretrained_summarizer import DistilBartSummarizer
import warnings
import time

from unidecode import unidecode
warnings.filterwarnings("ignore")

bart_cnn = BartSummarizer('cnn')
name_base = 'match_sticks'

with open(f"{name_base}.txt", 'r') as f:
    lines = f.readlines()

all_story = [unidecode(line.strip()) for line in lines]

json = []

sum_text = bart_cnn(all_story)

for og, sum in zip(all_story, sum_text):
    sum = sum['summary_text']
    json.append({'original': og, 'summary': sum})


Your max_length is set to 142, but you input_length is only 132. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 142, but you input_length is only 93. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 142, but you input_length is only 85. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 142, but you input_length is only 118. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)


In [3]:
from core.serialization import write_json
from pathlib import Path
seed = 5
root_dir = Path('/deep2/u/yma42/StableDiffusionProject/docs/') / f'{name_base}_seed{seed}'
root_dir.mkdir(parents=True, exist_ok=True)
write_json(json, str(root_dir/'meta.json'))

In [10]:
with open("rapunzel.txt", 'r') as f:
    lines = f.readlines()

# Eval

In [16]:
import rouge
import numpy as np
from lsa_summarizer import LsaSummarizer
import pandas as pd

import datasets
from tqdm import tqdm

dataset = datasets.load_dataset("cnn_dailymail", '3.0.0')
lst_dics = [dic for dic in dataset["test"]]
dtf = pd.DataFrame(lst_dics).rename(columns={"article":"text", 
      "highlights":"y"})[["text","y"]]

def evaluate_summary(y_test, predicted, verbose=False):    
    rouge_score = rouge.Rouge()    
    scores = rouge_score.get_scores(y_test, predicted, avg=True)       
    score_1 = scores['rouge-1']['f']
    score_2 = scores['rouge-2']['f']
    score_L = scores['rouge-l']['f']
    if verbose:
        print("rouge1:", score_1, "| rouge2:", score_2, "| rougeL:", score_2, "--> avg rouge:", round(np.mean( [score_1,score_2,score_L]), 2))
    return score_1, score_2, score_L

lsa = LsaSummarizer()

r1, r2, rl = 0, 0, 0
for text, y in tqdm(zip(dtf['text'], dtf['y']), total=len(dtf['text'])):
    pred = lsa(text, 1)[0]
    rouge_score = evaluate_summary(y, pred)
    r1 += rouge_score[0]
    r2 += rouge_score[1]
    rl += rouge_score[2]

print(f'rouge1: {r1/len(dtf["text"])}')
print(f'rouge2: {r2/len(dtf["text"])}')
print(f'rougeL: {rl/len(dtf["text"])}')

    

Found cached dataset cnn_dailymail (/deep2/u/yma42/files/cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 11490/11490 [02:06<00:00, 91.02it/s]

rouge1: 0.20485466384226367
rouge2: 0.05031070289889757
rougeL: 0.17852566935536796


In [14]:
len(dtf['text'])

11490

In [13]:
pred

'While a preliminary examination is not a formal investigation, it allows the court to review evidence and determine whether to investigate suspects on both sides.'